In [1]:
#lets start here

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

%matplotlib inline

In [2]:
#
# TRY NUMBER 3
#
from bs4 import BeautifulSoup
import requests as req

#
# SET DATE HERE 
#
date = '2019-08-10'
resp = req.get("https://www.domain.com.au/auction-results/sydney/" + date)

soup = BeautifulSoup(resp.text, 'lxml')

print(soup.prettify()) 

<!DOCTYPE html>
<html>
 <head>
  <link href="https://static.domain.com.au/domain/favicon.ico" rel="shortcut icon"/>
  <link href="https://static.domain.com.au/content/fe-static/@domain-group/fe-brary/css/fe-brary-globals-domain_8afccc11a77ff6ac.css" rel="stylesheet"/>
  <meta charset="utf-8"/>
  <meta content="en-au" http-equiv="content-language"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="//static.domain.com.au" rel="dns-prefetch"/>
  <link href="//rimh2.domainstatic.com.au" rel="dns-prefetch"/>
  <script>
   (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
                (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
                m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
                })(window,document,'script','https://www.g

In [7]:
#
#for tag in soup.find_all("article"):
#    print("{0}: {1}".format(tag.name, tag.text))

tags = soup.find_all("article")  # there is one article per suburb,many properties per article
#print(tags[0])

#df = pd.DataFrame(data, columns = ['Name', 'Age'])
df = pd.DataFrame()
print("Number suburbs", len(tags))

Number suburbs 140


In [8]:
once = False
j = 0
# Suburb name starts with h3 
for sid in range(len(tags)):
    if (once == True):
        print(tags[sid])
        once=False
    suburb = tags[sid].find(class_="css-1czqru0").get_text(":")
    for i in range(10):
        try:
            properties = tags[sid].find_all(class_="css-1hnr0ia")[i].get_text()
            x = tags[sid].find_all(class_="css-1hnr0ia")[i]
#            print(x)
#            print(x.get('href'))
            #print(properties)
            howsold = tags[sid].find_all(class_="css-43wvni")[i].get_text(":")
            #print(howsold)
            desc = tags[sid].find_all(class_="css-dpwygs")[i].get_text(":")
            #print(desc)
            # Work out how much for in AUD
            aud = 0
            value = howsold.split(':')[1]
            if (value[-1:] == 'm'):
                aud = float(value[1:-1]) * 1000000
            if (value[-1:] == 'k'):
                aud = float(value[1:-1]) * 1000
#            newrow = [ '18/7/2020', 
#                      suburb.split(':')[0], 
#                      properties,
#                      howsold.split(':')[0], 
#                      howsold.split(':')[1],
#                      aud,
#                      desc.split(':')[0], 
#                      desc.split(':')[1].split(' ')[0], 
#                      x.get('href')
#                     ]
            newrow = { 'index': j,
                      'date':date, 
                      'suburb' : suburb.split(':')[0], 
                      'address' :properties,
                      'action' :howsold.split(':')[0], 
                      'value' : aud,
                      'profile': desc.split(':')[0], 
                      'bed' : desc.split(':')[1].split(' ')[0], 
                      'url' :x.get('href')
                     }
            #print(newrow)
            df = df.append([newrow])
            j += 1
        except:
            _

#df.rename(columns={'0':'date','1':'suburb','2':'address','3':'action', '4':'value', '5':'property', '6':'beds', '7':'url'})
#df.colunns = ['date', 'suburb']
df.set_index('index', inplace=True)
print("Number properties", df.shape)


Number properties (163, 8)


In [9]:
import re
df.at[0,"bath"] = np.NaN
for hid in df.T:
#for hid in [82]:
#
#
# Lets read data for a particular house
#
#    print(df["bath"][hid])
    hurl = df['url'][hid]
    print(hid, "url=", hurl)
    if (df["bath"][hid] != df["bath"][hid]):
#        time.sleep(np.random.random_sample()*15)
        respp = req.get(hurl)

        soupp = BeautifulSoup(respp.text, 'lxml')
        try:
            photos = soupp.find_all(class_="css-e6s3qy")[0].get_text()
        except:
            photos = np.NaN
        #print(photos)

        #print(soupp.prettify())
        try:
            tags = soupp.find_all(class_="css-1rzse3v")[0].get_text()
            #print(tags)
            bed = float(soupp.find_all(class_="css-1rzse3v")[0].get_text()[0])
            bath = float(soupp.find_all(class_="css-1rzse3v")[1].get_text()[0])
            #print("bed=", bed, "bath=", bath)
            try:
                cars = float(soupp.find_all(class_="css-1rzse3v")[2].get_text()[0])
                # print("cars=",soupp.find_all(class_="css-1rzse3v")[2].get_text()[0])
            except:
                cars = np.NaN
            try:
                size = soupp.find_all(class_="css-1rzse3v")[3].get_text()
                # print("size=", size)
                if ("m" in size):
                    size = int(re.sub("[^0-9]", "", soupp.find_all(class_="css-1rzse3v")[3].get_text()))
                else:
                    size = np.NaN
            except:
                size = np.NaN
        except:
            bath = np.NaN
            cars = np.NaN
            size= np.NaN
            bed = np.NaN

        print(hid, "=", bed, bath,cars,size, photos)
        df.at[hid, 'bath'] = bath
        df.at[hid, 'car'] = cars
        df.at[hid, 'size'] = size
        df.at[hid, 'photo'] = photos
    else:
        print("Already done")
    

0 url= https://www.domain.com.au/34-alexander-street-alexandria-nsw-2015-2015446486
0 = 3.0 1.0 nan nan 10
1 url= https://www.domain.com.au/111-albion-street-annandale-nsw-2038-2015453418
1 = 2.0 1.0 nan 107 14
2 url= https://www.domain.com.au/23-albion-street-annandale-nsw-2038-2015457266
2 = 2.0 1.0 nan 158 12
3 url= https://www.domain.com.au/145-nelson-street-annandale-nsw-2038-2015456435
3 = 2.0 1.0 nan nan 7
4 url= https://www.domain.com.au/17-goodwin-avenue-ashfield-nsw-2131-2015432622
4 = 3.0 2.0 2.0 nan 14
5 url= https://www.domain.com.au/3-milton-lane-ashfield-nsw-2131-2015449583
5 = 3.0 2.0 1.0 341 10
6 url= https://www.domain.com.au/45-jackson-street-balgowlah-nsw-2093-2015458647
6 = 3.0 1.0 3.0 476 14
7 url= https://www.domain.com.au/d3-1-buchanan-street-balmain-nsw-2041-2015450805
7 = 2.0 2.0 1.0 nan 13
8 url= https://www.domain.com.au/186-darling-street-balmain-nsw-2041-2015467978
8 = 3.0 1.0 nan nan 10
9 url= https://www.domain.com.au/4-ennis-street-balmain-nsw-2041-2015

77 = 3.0 2.0 1.0 253 5
78 url= https://www.domain.com.au/9-31-39-elamang-avenue-kirribilli-nsw-2061-2015458413
78 = 3.0 2.0 1.0 nan 10
79 url= https://www.domain.com.au/33-dampier-street-kurnell-nsw-2231-2015456210
79 = 4.0 1.0 1.0 5501 7
80 url= https://www.domain.com.au/127-bestic-street-kyeemagh-nsw-2216-2015115735
80 = 5.0 2.0 1.0 nan 9
81 url= https://www.domain.com.au/5-owen-avenue-kyeemagh-nsw-2216-2015466957
81 = 4.0 3.0 3.0 474 5
82 url= https://www.domain.com.au/50-302-burns-bay-road-lane-cove-nsw-2066-2015434458
82 = 2.0 1.0 1.0 nan 9
83 url= https://www.domain.com.au/59-sutherland-street-lane-cove-nsw-2066-2015452875
83 = 4.0 2.0 1.0 600 5
84 url= https://www.domain.com.au/7-52-helen-street-lane-cove-nsw-2066-2015452479
84 = 2.0 1.0 1.0 nan 6
85 url= https://www.domain.com.au/26-mooney-street-lane-cove-nsw-2066-2015452510
85 = 3.0 1.0 1.0 8347 10
86 url= https://www.domain.com.au/18-1-5-stokes-street-lane-cove-nsw-2066-2015474721
86 = 2.0 1.0 1.0 nan 8
87 url= https://www.d

154 = 2.0 1.0 2.0 808 8
155 url= https://www.domain.com.au/631-2c-munderah-street-wahroonga-nsw-2076-2015431310
155 = 2.0 2.0 1.0 nan 8
156 url= https://www.domain.com.au/312-100-bay-road-waverton-nsw-2060-2015456233
156 = 1.0 1.0 1.0 nan 7
157 url= https://www.domain.com.au/9-kooloona-crescent-west-pymble-nsw-2073-2015436814
157 = 3.0 2.0 1.0 925 8
158 url= https://www.domain.com.au/38-the-comenarra-parkway-west-pymble-nsw-2073-2015458645
158 = 4.0 2.0 2.0 942 5
159 url= https://www.domain.com.au/37-kirkstone-road-wheeler-heights-nsw-2097-2015438801
159 = 5.0 3.0 2.0 nan 27
160 url= https://www.domain.com.au/158-2-artarmon-road-willoughby-nsw-2068-2015455905
160 = 2.0 1.0 1.0 nan 8
161 url= https://www.domain.com.au/4-29b-shirley-road-wollstonecraft-nsw-2065-2015452908
161 = 2.0 1.0 2.0 nan 7
162 url= https://www.domain.com.au/201-attunga-road-yowie-bay-nsw-2228-2015415931
162 = 5.0 3.0 2.0 nan 13


In [7]:
df


,date,suburb,address,action,value,profile,bed,url,bath,car,size,photo
index,,,,,,,,,,,,
0,2019-08-10,Alexandria,34 Alexander St,Sold prior to auction,1551000.0,House,3,https://www.domain.com.au/34-alexander-street-...,1.0,NaN,NaN,10
1,2019-08-10,Annandale,111 Albion St,Sold,0.0,House,2,https://www.domain.com.au/111-albion-street-an...,1.0,NaN,107.0,14
2,2019-08-10,Annandale,23 Albion St,Sold,1360000.0,House,2,https://www.domain.com.au/23-albion-street-ann...,1.0,NaN,158.0,12
3,2019-08-10,Annandale,145 Nelson St,Sold prior to auction,1325000.0,House,2,https://www.domain.com.au/145-nelson-street-an...,1.0,NaN,NaN,7
4,2019-08-10,Ashfield,17 Goodwin Av,Sold,1541000.0,House,3,https://www.domain.com.au/17-goodwin-avenue-as...,2.0,2.0,NaN,14
...,...,...,...,...,...,...,...,...,...,...,...,...
158,2019-08-10,West Pymble,38 The Comenarra Pky,Sold prior to auction,1530000.0,House,4,https://www.domain.com.au/38-the-comenarra-par...,2.0,2.0,942.0,5
159,2019-08-10,Wheeler Heights,37 Kirkstone Rd,Sold,1472500.0,House,5,https://www.domain.com.au/37-kirkstone-road-wh...,3.0,2.0,NaN,27
160,2019-08-10,Willoughby,158/2 Artarmon Rd,Sold,955000.0,Unit,2,https://www.domain.com.au/158-2-artarmon-road-...,1.0,1.0,NaN,8


In [8]:
#
#
# Output as CSV
#
file_name="Sydney-" + date + ".csv"
df.to_csv(file_name, sep=',', encoding='utf-8')